In [1]:
import pandas as pd
from ast import literal_eval
from datetime import datetime
import timestring
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
dirty_data = pd.read_csv('Group017_dirty_data.csv')
outlier_data = pd.read_csv('Group017_outlier_data.csv')

In [3]:
dirty_data.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,496.215100,-30.753928,143.504939,0.122000,8.821536,13.909427
std,285.920345,25.336588,16.298470,0.327614,1.721933,2.492537
min,29.500000,-37.833482,-37.809475,0.000000,3.554000,4.416368
25%,268.750000,-37.818139,144.949985,0.000000,7.828750,12.549057
50%,438.000000,-37.811960,144.963857,0.000000,8.801500,13.974469
75%,670.050000,-37.803914,144.982768,0.000000,9.892000,15.612416
max,1411.000000,144.972017,145.019877,1.000000,14.095000,21.316415


### 1. date

In [3]:
def todate(row):
    datetimestr = row.date
    date = ''
    try:
          date = str(datetime.strptime(datetimestr, '%Y-%m-%d' ))[:10]

#         date = datetime.strptime(datetimestr, '%d-%m-%y %H:%M:%S')
    except: 
        try: 
            date = str(datetime.strptime(datetimestr, '%d-%m-%y'))[0:10]
        except: 
            try:
                date = str(datetime.strptime(datetimestr, '%Y-%d-%m'))[0:10]
            except:
                try:
                    date = str(datetime.strptime(datetimestr, '%d-%m-%Y'))[0:10]
                except:
                    print(row)
                    
    return date

#     except: 
#         print('exception')
    

In [4]:
dirty_data.date =  dirty_data.apply(todate, axis = 1)

In [5]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Lunch,NS,"[('Fish&Chips', 3), ('Salmon', 1), ('Shrimp', 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[('Pancake', 10), ('Cereal', 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[('Eggs', 7), ('Pancake', 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[('Coffee', 7), ('Pancake', 1)]",76.75,37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[('Chicken', 8), ('Steak', 4), ('Fries', 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[('Chicken', 6), ('Salad', 3), ('Steak', 1), (...",613.60,-37.809597,144.953554,0,8.723,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[('Cereal', 2), ('Eggs', 8), ('Coffee', 2), ('...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[('Fries', 9), ('Burger', 1), ('Salad', 4), ('...",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,tp,"[('Salad', 10), ('Chicken', 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,ns,"[('Chicken', 2), ('Burger', 6), ('Salad', 4), ...",693.80,-37.821505,144.969280,0,7.772,11.211580


### 2. order_type

In [6]:
for row in dirty_data.itertuples():
    if(row.order_type == 'Breakfast'):
        if(int(row.time[0:2])>12):
                print(row)
        elif(int(row.time[0:2])==12 and int(row.time[3:5])>1):
                print(row)
        elif(int(row.time[0:2])==12 and int(row.time[3:5])==0 and int(row.time[6:8])>1):
                print(row)
                
                
                
                
                
    if(row.order_type == 'Lunch'):
            if(int(row.time[0:2])<12 or int(row.time[0:2])>16):
                    print(row)
            elif(int(row.time[0:2])==16 and int(row.time[3:5])>1):
                    print(row)
            elif(int(row.time[0:2])==16 and int(row.time[3:5])==0 and int(row.time[6:8])>1):
                    print(row)
                
                
                
                
    if(row.order_type == 'Dinner'):
            if(int(row.time[0:2]))<16:
                    print(row)
#         elif(int(row.time[0:2])==16 and int(row.time[3:5])>1):
#                 print(row)
#         elif(int(row.time[0:2])==16 and int(row.time[3:5])==0 and int(row.time[6:8])>1):
#                 print(row)

Pandas(Index=0, order_id='ORDC03597', date='2018-10-28', time='17:58:18', order_type='Lunch', branch_code='NS', order_items="[('Fish&Chips', 3), ('Salmon', 1), ('Shrimp', 4)]", order_price=362.0, customer_lat=-37.8107373, customer_lon=144.9462583, _10=0, distance_to_customer_KM=9.037, delivery_fee=17.0678333)
Pandas(Index=20, order_id='ORDK08729', date='2018-10-20', time='17:38:01', order_type='Lunch', branch_code='BK', order_items="[('Fish&Chips', 6), ('Pasta', 9)]", order_price=457.5, customer_lat=-37.8081652, customer_lon=144.9729806, _10=0, distance_to_customer_KM=6.545, delivery_fee=15.777474100000001)
Pandas(Index=39, order_id='ORDX08730', date='2018-11-17', time='18:38:52', order_type='Lunch', branch_code='BK', order_items="[('Salmon', 4), ('Shrimp', 9)]", order_price=650.0, customer_lat=-37.816865, customer_lon=144.9289353, _10=0, distance_to_customer_KM=11.828, delivery_fee=21.316414899999998)
Pandas(Index=48, order_id='ORDC06711', date='2018-04-23', time='11:43:05', order_typ

In [7]:
def changeorder_type(row):
        
        if int(row.time[0:2])>8 and int(row.time[0:2])<12:
            return 'Breakfast'
        elif int(row.time[0:2])==12 and int(row.time[3:5])==0 and int(row.time[6:8])==0:
            return 'Breakfast'
        elif int(row.time[0:2])>=12 and int(row.time[0:2])<16:
            return 'Lunch'
        elif int(row.time[0:2])==16 and int(row.time[3:5])==0 and int(row.time[6:8])==0:
            return 'Lunch'
        elif int(row.time[0:2])>=16:
            return 'Dinner'


In [8]:
dirty_data.order_type = dirty_data.apply(changeorder_type, axis = 1)

In [9]:
#Checking
for row in dirty_data.itertuples():
    if(row.order_type == 'Breakfast'):
        if(int(row.time[0:2])>12):
                print(row)
        elif(int(row.time[0:2])==12 and int(row.time[3:5])>1):
                print(row)
        elif(int(row.time[0:2])==12 and int(row.time[3:5])==0 and int(row.time[6:8])>1):
                print(row)
                
                
                
                
                
    if(row.order_type == 'Lunch'):
            if(int(row.time[0:2])<12 or int(row.time[0:2])>16):
                    print(row)
            elif(int(row.time[0:2])==16 and int(row.time[3:5])>1):
                    print(row)
            elif(int(row.time[0:2])==16 and int(row.time[3:5])==0 and int(row.time[6:8])>1):
                    print(row)
                
                
                
                
    if(row.order_type == 'Dinner'):
            if(int(row.time[0:2]))<16:
                    print(row)


In [10]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[('Fish&Chips', 3), ('Salmon', 1), ('Shrimp', 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[('Pancake', 10), ('Cereal', 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[('Eggs', 7), ('Pancake', 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[('Coffee', 7), ('Pancake', 1)]",76.75,37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[('Chicken', 8), ('Steak', 4), ('Fries', 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[('Chicken', 6), ('Salad', 3), ('Steak', 1), (...",613.60,-37.809597,144.953554,0,8.723,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[('Cereal', 2), ('Eggs', 8), ('Coffee', 2), ('...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[('Fries', 9), ('Burger', 1), ('Salad', 4), ('...",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,tp,"[('Salad', 10), ('Chicken', 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,ns,"[('Chicken', 2), ('Burger', 6), ('Salad', 4), ...",693.80,-37.821505,144.969280,0,7.772,11.211580


##### Branch Code

In [11]:
dirty_data.branch_code.value_counts()

TP    172
NS    155
BK    148
ns     12
bk      8
tp      5
Name: branch_code, dtype: int64

In [12]:
dirty_data.branch_code.replace({'ns':'NS', 'bk':'BK', 'tp':'TP'}, inplace = True)

In [13]:
dirty_data.branch_code.value_counts()

TP    177
NS    167
BK    156
Name: branch_code, dtype: int64

In [14]:
for row in dirty_data.itertuples():
    if((row.order_id[:4]=='ORDA' or row.order_id[:4]=='ORDK' or row.order_id[:4]=='ORDX') and row.branch_code != 'BK'):
        print(row)
    elif((row.order_id[:4]=='ORDB' or row.order_id[:4]=='ORDJ' or row.order_id[:4]=='ORDY')and row.branch_code != 'TP') :
        print(row)
    elif((row.order_id[:4]=='ORDC' or row.order_id[:4]=='ORDI' or row.order_id[:4]=='ORDZ')and row.branch_code != 'NS') :
        print(row)


Pandas(Index=8, order_id='ORDK09302', date='2018-05-19', time='12:33:48', order_type='Lunch', branch_code='TP', order_items="[('Salad', 10), ('Chicken', 3)]", order_price=268.0, customer_lat=-37.8134003, customer_lon=144.9534121, _10=0, distance_to_customer_KM=8.488, delivery_fee=16.8632784)
Pandas(Index=9, order_id='ORDY06134', date='2018-07-16', time='14:25:21', order_type='Lunch', branch_code='NS', order_items="[('Chicken', 2), ('Burger', 6), ('Salad', 4), ('Fries', 5), ('Steak', 7)]", order_price=693.8, customer_lat=-37.8215051, customer_lon=144.9692796, _10=0, distance_to_customer_KM=7.772, delivery_fee=11.211580300000001)
Pandas(Index=25, order_id='ORDA10400', date='2018-06-19', time='13:34:38', order_type='Lunch', branch_code='NS', order_items="[('Steak', 3), ('Salad', 5), ('Burger', 7), ('Chicken', 1)]", order_price=470.0, customer_lat=-37.8240055, customer_lon=145.0020524, _10=0, distance_to_customer_KM=5.194, delivery_fee=10.780248)
Pandas(Index=31, order_id='ORDX07973', date

In [15]:
outlier_data['order_id']= outlier_data.order_id.apply(lambda x:x[0:4])

In [16]:
outlier_data.groupby(['order_id', 'branch_code']).size()

order_id  branch_code
ORDA      BK             46
ORDB      TP             57
ORDC      NS             52
ORDI      NS             54
ORDJ      TP             60
ORDK      BK             66
ORDX      BK             52
ORDY      TP             61
ORDZ      NS             52
dtype: int64

In [17]:
def change_branch_code(row):
    if(row.order_id[:4]=='ORDA' or row.order_id[:4]=='ORDK' or row.order_id[:4]=='ORDX'):
        return 'BK'
    elif(row.order_id[:4]=='ORDB' or row.order_id[:4]=='ORDJ' or row.order_id[:4]=='ORDY'):
        return 'TP'
    else: 
        return 'NS'

In [18]:
dirty_data.branch_code = dirty_data.apply(change_branch_code, axis = 1)

In [19]:
dirty_data.branch_code.value_counts()

TP    184
NS    161
BK    155
Name: branch_code, dtype: int64

In [20]:
#checking
for row in dirty_data.itertuples():
    if((row.order_id[:4]=='ORDA' or row.order_id[:4]=='ORDK' or row.order_id[:4]=='ORDX') and row.branch_code != 'BK'):
        print(row)
    elif((row.order_id[:4]=='ORDB' or row.order_id[:4]=='ORDJ' or row.order_id[:4]=='ORDY')and row.branch_code != 'TP') :
        print(row)
    elif((row.order_id[:4]=='ORDC' or row.order_id[:4]=='ORDI' or row.order_id[:4]=='ORDZ')and row.branch_code != 'NS') :
        print(row)

##### Order Items

In [21]:
dirty_data.order_items = dirty_data.order_items.apply(lambda x: literal_eval(x))

In [22]:
outlier_data.order_items = outlier_data.order_items.apply(lambda x: literal_eval(x))

In [23]:
lunchitems = []
breakfastitems = []
dinneritems = []

In [24]:
for item in np.unique(outlier_data[outlier_data.order_type == 'Lunch'].order_items):
    for food in item: 
        lunchitems.append(food[0])

In [25]:
lunchitems = np.unique(lunchitems)

In [26]:
for item in np.unique(outlier_data[outlier_data.order_type == 'Dinner'].order_items):
    for food in item: 
        dinneritems.append(food[0])
dinneritems = np.unique(dinneritems)



In [27]:
for item in np.unique(outlier_data[outlier_data.order_type == 'Breakfast'].order_items):
    for food in item: 
        breakfastitems.append(food[0])
breakfastitems = np.unique(breakfastitems)

In [28]:
print(lunchitems)
print(breakfastitems)
print(dinneritems)

['Burger' 'Chicken' 'Fries' 'Salad' 'Steak']
['Cereal' 'Coffee' 'Eggs' 'Pancake']
['Fish&Chips' 'Pasta' 'Salmon' 'Shrimp']


In [29]:
def fix_order_items(row):
    order_items = row.order_items
    method = row.order_type
#     print(method)
    if(method == 'Lunch'):
        for item in order_items:
            if item[0] not in lunchitems:
                order_items.remove(item)
#             return order_items
    if(method == 'Dinner'):
        for item in order_items:
            if item[0] not in dinneritems:
                order_items.remove(item)
#             return order_items   
    if(method == 'Breakfast'):
#         print(order_items)
        for item in order_items:
            if item[0] in breakfastitems:
                pass
            else:
                order_items.remove(item)
                print(item)
#                 print(order_items)
    return order_items

In [30]:
dirty_data.order_items = dirty_data.apply(fix_order_items, axis = 1)

('Pasta', 5)
('Fries', 1)
('Salad', 6)
('Salmon', 5)
('Shrimp', 9)
('Fish&Chips', 2)
('Steak', 9)
('Steak', 5)
('Salad', 9)
('Burger', 9)
('Fish&Chips', 6)
('Fish&Chips', 8)


In [31]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,8.723,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580


##### Order Price

In [38]:
dirty_data[(dirty_data.order_price>1000) & (dirty_data.order_type=='Breakfast')]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [32]:
a = []
b = []
c=[]
d=[]
f=[]
e = []
g=[]

for data in outlier_data[outlier_data.order_type=='Breakfast'].head(10).itertuples():
    if(len(data.order_items)>3):
        print(sorted(data.order_items))
#         print(data.order_price)
        for x in sorted(data.order_items): 
            a.append(x[1])
        e.append(data.order_price)

print(a)
b = a[4:8]
c = a[8:12]
d = a[12:16]
a = a[:4]
# print(a)

# print(b)
# print(c)
# print(d)
print(e)

x = [a,b,c,d]
solvedeq= np.linalg.solve(x,e)
breakfastitems = list(breakfastitems)
# print(lunchitems)
breakfast_prices = dict()
for item in sorted(breakfastitems):
    breakfast_prices[item]=solvedeq[breakfastitems.index(item)]
        
print(breakfast_prices)


[('Cereal', 5), ('Coffee', 9), ('Eggs', 9), ('Pancake', 2)]
[('Cereal', 5), ('Coffee', 10), ('Eggs', 8), ('Pancake', 4)]
[('Cereal', 1), ('Coffee', 10), ('Eggs', 7), ('Pancake', 4)]
[('Cereal', 7), ('Coffee', 5), ('Eggs', 3), ('Pancake', 1)]
[5, 9, 9, 2, 5, 10, 8, 4, 1, 10, 7, 4, 7, 5, 3, 1]
[419.0, 453.0, 347.0, 274.75]
{'Cereal': 21.0, 'Coffee': 7.500000000000006, 'Eggs': 22.0, 'Pancake': 24.249999999999982}


In [33]:
a = []
b = []
c=[]
d=[]
f=[]
e = []

for data in outlier_data[outlier_data.order_type=='Lunch'].head(20).itertuples():
    if(len(data.order_items)>4):
        print(sorted(data.order_items))
#         print(data.order_price)
        for x in sorted(data.order_items):
#             print(x)
            a.append(x[1])
        e.append(data.order_price)

# print(a)
b = a[5:10]
c = a[10:15]
d = a[15:20]
f = a[20:25]
# print(a)
a = a[:5]

# print(b)
# print(c)
# print(d)
# print(f)
# print(e)
x = [a,b,c,d,f]
solvedeq= np.linalg.solve(x,e)
lunchitems = list(lunchitems)
# print(lunchitems)
lunch_prices = dict()
for item in sorted(lunchitems):
    lunch_prices[item]=solvedeq[lunchitems.index(item)]
        
print(lunch_prices)


[('Burger', 1), ('Chicken', 9), ('Fries', 8), ('Salad', 10), ('Steak', 3)]
[('Burger', 7), ('Chicken', 3), ('Fries', 6), ('Salad', 7), ('Steak', 1)]
[('Burger', 7), ('Chicken', 2), ('Fries', 4), ('Salad', 6), ('Steak', 4)]
[('Burger', 1), ('Chicken', 10), ('Fries', 2), ('Salad', 1), ('Steak', 2)]
[('Burger', 10), ('Chicken', 5), ('Fries', 8), ('Salad', 8), ('Steak', 7)]
{'Burger': 30.999999999999982, 'Chicken': 31.99999999999999, 'Fries': 12.000000000000032, 'Salad': 17.19999999999998, 'Steak': 45.00000000000002}


In [34]:
a = []
b = []
c=[]
d=[]
f=[]
e = []
g=[]

for data in outlier_data[outlier_data.order_type=='Dinner'].head(11).itertuples():
    if(len(data.order_items)>3):
        print(sorted(data.order_items))
#         print(data.order_price)
        for x in sorted(data.order_items): 
            a.append(x[1])
        e.append(data.order_price)

# print(a)
b = a[4:8]
c = a[8:12]
d = a[12:16]
a = a[:4]
# print(a)

# print(b)
# print(c)
# print(d)
# print(e)

x = [a,b,c,d]
solvedeq= np.linalg.solve(x,e)
dinneritems = list(dinneritems)
# print(lunchitems)
dinner_prices = dict()
for item in sorted(dinneritems):
    dinner_prices[item]=solvedeq[dinneritems.index(item)]
        
print(dinner_prices)


[('Fish&Chips', 8), ('Pasta', 3), ('Salmon', 6), ('Shrimp', 2)]
[('Fish&Chips', 6), ('Pasta', 1), ('Salmon', 5), ('Shrimp', 8)]
[('Fish&Chips', 9), ('Pasta', 9), ('Salmon', 9), ('Shrimp', 3)]
[('Fish&Chips', 4), ('Pasta', 10), ('Salmon', 6), ('Shrimp', 4)]
{'Fish&Chips': 35.00000000000004, 'Pasta': 27.500000000000018, 'Salmon': 40.999999999999936, 'Shrimp': 54.00000000000001}


In [35]:
def change_prices(row):
    try: 
        price = 0
        if(row.order_type == 'Breakfast'):
            for item in row.order_items:
                price += breakfast_prices[item[0]]*item[1]
#         print(price)
#         print('hello')
#         print(row.order_price)
    
        if(row.order_type == 'Lunch'):
            for item in row.order_items:
                price += lunch_prices[item[0]]*item[1]
#         print(price)
#         print('hello')
#         print(row.order_price)
    
        if(row.order_type == 'Dinner'):
            for item in row.order_items:
                price += dinner_prices[item[0]]*item[1]
#         print(price)
#         print('hello')
#         print(row.order_price)
#         print(price)
        return price
    except:
        print(row)


In [36]:
dirty_data.order_price = dirty_data.apply(change_prices, axis = 1)

In [37]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,8.723,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580


##### Dealing with lat long

In [39]:
dirty_data.customer_lat.describe()

count    500.000000
mean     -30.753928
std       25.336588
min      -37.833482
25%      -37.818139
50%      -37.811960
75%      -37.803914
max      144.972017
Name: customer_lat, dtype: float64

In [41]:
dirty_data[dirty_data.customer_lat>40]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
273,ORDZ03820,2018-03-18,10:32:06,Breakfast,NS,"[(Cereal, 6), (Pancake, 8)]",320.0,144.972017,-37.809475,0,7.200,14.640792
384,ORDB01803,2018-09-22,16:47:19,Dinner,TP,"[(Fish&Chips, 3), (Pasta, 4)]",215.0,144.971437,-37.800578,0,10.062,15.658808
385,ORDA00559,2018-02-08,09:10:59,Breakfast,BK,"[(Coffee, 5), (Cereal, 7), (Pancake, 4), (Eggs...",435.5,144.959419,-37.806323,0,7.749,12.956898
406,ORDK07541,2018-01-16,08:30:25,None,BK,"[(Pancake, 7), (Eggs, 8), (Coffee, 4), (Cereal...",0.0,144.928088,-37.805151,1,10.738,7.806882


In [46]:
def change_customer_lat_lon(row):
        lat = row.customer_lat
        lon = row.customer_lon
        if(lat>0):
            if(lat>40):
                temp = lat
                lat = lon
                lon = temp
            else: 
                lat = lat * -1
        if(lon<0):
            lon = lon *-1
        row.customer_lat = lat
        row.customer_lon = lon
        return row

In [47]:
dirty_data = dirty_data.apply(change_customer_lat_lon, axis = 1)

In [48]:
dirty_data[dirty_data.customer_lat>40]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [49]:
dirty_data[dirty_data.customer_lat>0]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [50]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,-37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,8.723,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580


##### DIjikistra algorithm

In [62]:
branches = pd.read_csv('branches.csv')
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')

In [63]:
branch_nodes=pd.merge(branches, nodes, how='left', left_on=['branch_lat','branch_lon'], right_on = ['lat','lon'])
branch_nodes

,branch_code,branch_name,branch_lat,branch_lon,node,lat,lon
0,NS,Nickolson,-37.773803,144.983647,2455254505,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716,1390575046,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450,1889485053,-37.815834,145.046450


In [64]:
branch_nodes=branch_nodes.drop(['lat', 'lon'], axis=1)

In [65]:
branch_nodes.rename(columns={'node':'branch_node'}, inplace=True)

In [66]:
dirty_data=pd.merge(dirty_data, nodes, how='left', left_on=['customer_lat','customer_lon'], right_on = ['lat','lon'])
dirty_data=dirty_data.drop(['lat', 'lon'], axis=1)
dirty_data.rename(columns={'node':'customer_node'}, inplace=True)
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833,4655129025
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678,6167359495
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708,130040449
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,-37.804479,144.973207,0,9.657,11.680867,4421481865
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645,368380518
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,8.723,12.363141,387266341
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871,2465501923
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657,4637139147
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278,163364742
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580,3925153646


In [67]:
dirty_data=pd.merge(dirty_data, branch_nodes, how='left', left_on=['branch_code'], right_on = ['branch_code'])

In [68]:
nodes_list=list(nodes['node'])

In [69]:
edges_list=list(zip(edges.u, edges.v, edges['distance(m)']))

In [70]:
import networkx as nx

G=nx.Graph()
G.add_nodes_from(nodes_list)
G.add_weighted_edges_from(edges_list)


In [71]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node,branch_name,branch_lat,branch_lon,branch_node
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833,4655129025,Nickolson,-37.773803,144.983647,2455254505
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678,6167359495,Bakers,-37.815834,145.046450,1889485053
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708,130040449,Thompson,-37.861835,144.905716,1390575046
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,-37.804479,144.973207,0,9.657,11.680867,4421481865,Thompson,-37.861835,144.905716,1390575046
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645,368380518,Bakers,-37.815834,145.046450,1889485053
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,8.723,12.363141,387266341,Thompson,-37.861835,144.905716,1390575046
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871,2465501923,Nickolson,-37.773803,144.983647,2455254505
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657,4637139147,Nickolson,-37.773803,144.983647,2455254505
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278,163364742,Bakers,-37.815834,145.046450,1889485053
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580,3925153646,Thompson,-37.861835,144.905716,1390575046


In [72]:
source_list=list(dirty_data['branch_node'])
target_list=list(dirty_data['customer_node'])

In [73]:
dist=list()
for i in range(len(source_list)):
    dist.append((nx.dijkstra_path_length(G, source_list[i], target_list[i]))/1000)
len(dist)

500

In [75]:
dirty_data['distance_to_customer_KM']=dist

In [76]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,customer_node,branch_name,branch_lat,branch_lon,branch_node
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833,4655129025,Nickolson,-37.773803,144.983647,2455254505
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678,6167359495,Bakers,-37.815834,145.046450,1889485053
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708,130040449,Thompson,-37.861835,144.905716,1390575046
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,-37.804479,144.973207,0,9.657,11.680867,4421481865,Thompson,-37.861835,144.905716,1390575046
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645,368380518,Bakers,-37.815834,145.046450,1889485053
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,9.039,12.363141,387266341,Thompson,-37.861835,144.905716,1390575046
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871,2465501923,Nickolson,-37.773803,144.983647,2455254505
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657,4637139147,Nickolson,-37.773803,144.983647,2455254505
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278,163364742,Bakers,-37.815834,145.046450,1889485053
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580,3925153646,Thompson,-37.861835,144.905716,1390575046


In [77]:
dirty_data=dirty_data.drop(['branch_node', 'branch_lon', 'branch_lat', 'branch_name', 'customer_node' ], axis=1)

### customerHasloyalty

In [78]:
dirty_data

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDC03597,2018-10-28,17:58:18,Dinner,NS,"[(Fish&Chips, 3), (Salmon, 1), (Shrimp, 4)]",362.00,-37.810737,144.946258,0,9.037,17.067833
1,ORDA09900,2018-03-25,10:42:15,Breakfast,BK,"[(Pancake, 10), (Cereal, 3)]",305.50,-37.810454,144.972977,0,6.685,13.772678
2,ORDB03994,2018-02-21,09:41:24,Breakfast,TP,"[(Eggs, 7), (Pancake, 3)]",226.75,-37.804668,144.935219,0,10.857,12.446708
3,ORDB07951,2018-03-29,10:52:23,Breakfast,TP,"[(Coffee, 7), (Pancake, 1)]",76.75,-37.804479,144.973207,0,9.657,11.680867
4,ORDK06857,2018-07-06,14:05:04,Lunch,BK,"[(Chicken, 8), (Steak, 4), (Fries, 3)]",472.00,-37.798864,144.968687,0,7.806,13.760645
5,ORDB04446,2018-08-07,15:16:03,Lunch,TP,"[(Chicken, 6), (Salad, 3), (Steak, 1), (Fries,...",613.60,-37.809597,144.953554,0,9.039,12.363141
6,ORDI03577,2018-04-20,11:32:57,Breakfast,NS,"[(Cereal, 2), (Eggs, 8), (Coffee, 2), (Pancake...",378.50,-37.805672,144.957139,0,7.813,12.661871
7,ORDI06755,2018-07-22,14:35:29,Lunch,NS,"[(Fries, 9), (Burger, 1), (Salad, 4), (Steak, 1)]",252.80,-37.808617,144.948207,0,8.777,16.749657
8,ORDK09302,2018-05-19,12:33:48,Lunch,BK,"[(Salad, 10), (Chicken, 3)]",268.00,-37.813400,144.953412,0,8.488,16.863278
9,ORDY06134,2018-07-16,14:25:21,Lunch,TP,"[(Chicken, 2), (Burger, 6), (Salad, 4), (Fries...",693.80,-37.821505,144.969280,0,7.772,11.211580


In [82]:
dirty_data['customerHasloyalty?'].value_counts()

0    439
1     61
Name: customerHasloyalty?, dtype: int64